# Example Code to test the Pub_worm API

In [2]:
import sys
import inspect
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter
import os
import json

# Add pub_worm directory to the Python path
sys.path.append("/Users/dan/Code/Python/pub_worm")

from pub_worm.ncbi.entreze_api import EntrezAPI

# Find where EntrezAPI is being load from
module = inspect.getmodule(EntrezAPI)
if hasattr(module, "__file__"):
    file_path = module.__file__
    print("EntrezAPI imported from:", file_path)
else:
    print("Could not determine the file path.")

EntrezAPI imported from: /Users/dan/Code/Python/pub_worm/pub_worm/ncbi/entreze_api.py


In [3]:
# Example call to ePost and eSummary
data = ["10021351", "10022905", "10022914", "10022975", "10048487", "10048958", "10049162", "10049362", "10049567", "10049576", "10051671", "10051850", "10064800", "10066248"]

ncbi_api = EntrezAPI()
entreze_epost_result = ncbi_api.entreze_epost(data)

if 'WebEnv' in entreze_epost_result:
    entreze_summaries_result = ncbi_api.entreze_pmid_summaries(entreze_epost_result)
    pretty_data = json.dumps(entreze_summaries_result, indent=4)
    print(pretty_data)

[
    {
        "uid": "10021351",
        "issn": "0950-1991",
        "essn": "",
        "last_author": "Ueno N",
        "pmc_id": "",
        "title": "Regulation of body length and male tail ray pattern formation of Caenorhabditis elegans by a member of TGF-beta family.",
        "source": "Development"
    },
    {
        "uid": "10022905",
        "issn": "0270-7306",
        "essn": "1098-5549",
        "last_author": "Kimble J",
        "pmc_id": "PMC84011",
        "title": "The Caenorhabditis elegans sex determination gene mog-1 encodes a member of the DEAH-Box protein family.",
        "source": "Mol Cell Biol"
    },
    {
        "uid": "10022914",
        "issn": "0270-7306",
        "essn": "1098-5549",
        "last_author": "Schlessinger J",
        "pmc_id": "PMC84020",
        "title": "Identification of a novel family of targets of PYK2 related to Drosophila retinal degeneration B (rdgB) protein.",
        "source": "Mol Cell Biol"
    },
    {
        "uid": "10

In [11]:
# Emple code to Get full Articles from entrez/eutils
import requests
import time

def fetch_full_article(pmid):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {
        "db": "pmc",
        "id": pmid,
        "retmode": "xml"
    }

    response = requests.get(base_url, params=params)
    time.sleep(2)

    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to fetch article for PMID {pmid}. Status code: {response.status_code}")
        return None

# Example usage
pmid = "10022975"  # Replace with the PMID of the article you want to fetch
pmids = ["10021351", "10022905", "10022914", "10022975", "10048487", "10048958", "10049162", "10049362", "10049567", "10049576", "10051671", "10051850", "10064800", "10066248"]
for pmid in pmids:
    article_xml = fetch_full_article(pmid)
    if article_xml:
        with open(f"./output/article_{pmid}.xml", "w") as file:
            file.write(article_xml)


Failed to fetch article for PMID 10022914. Status code: 400
